In [1]:
from collections import Counter
import re
import os

dataset_dirs = ['./data/clean', './data/additional']  # Set the dataset directory
train_dir = './data/train'
val_dir = './data/val'

words = []
words_path = []

for dataset_dir in dataset_dirs:
    for root, dirs, files in os.walk(dataset_dir, topdown=True):
        for name in files:
            words.append(re.split(r'_|\.', name)[1])
            words_path.append(os.path.join(root, name))

In [3]:
words_counter = [['isnull', 0]]

for (key, count) in Counter(words).items():
    if count >= 40:
        words_counter.append([key, count])
    else:
        words_counter[0][1] += count

words_counter = dict(words_counter)

In [5]:
import shutil

if os.path.isdir(train_dir):
    shutil.rmtree(train_dir)
os.mkdir(train_dir)

if os.path.isdir(val_dir):
    shutil.rmtree(val_dir)
os.mkdir(val_dir)

wordset = set()
for key in words_counter:
    if key != 'isnull':
        wordset.add(key)
    os.mkdir(os.path.join(train_dir, key))
    os.mkdir(os.path.join(val_dir, key))

In [6]:
import random

from PIL import Image

random.shuffle(words_path)

pass_words = []

for i, path in enumerate(words_path):
    img = Image.open(path)
    current_word = re.split('_|\.', path)[-2]
    if not current_word in wordset:
        current_word = 'isnull'
    if Counter(pass_words)[current_word] < 0.8 * words_counter[current_word]:
        img.save(os.path.join(train_dir, current_word, path.split('/')[-1]))
    else:
        img.save(os.path.join(val_dir, current_word, path.split('/')[-1]))
    pass_words.append(current_word)